In [ ]:
import numpy as np
from torchvision.datasets import ImageFolder
from torchvision import transforms

In [ ]:
# 导入训练图片的路径
trans_compose = transforms.Compose([
        #转换为torch数据类型
        transforms.ToTensor(),
        # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
        # 调整图片大小
        transforms.Resize((100,100))

        ])

ds = ImageFolder(r'C:\Users\seven\Downloads\imgs\raw-img',trans_compose)
ds 

In [ ]:
ds.class_to_idx

In [ ]:
ds.imgs

In [ ]:
import torch

from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data import DataLoader
# 划分训练和测试的数据
train, test = random_split(ds, (25000, 1179) )
# 设定每次读取的数量
data_loader = torch.utils.data.DataLoader(dataset=train,batch_size=200, shuffle=True)
data_loader2 = torch.utils.data.DataLoader(dataset=test,batch_size=200, shuffle=True)


In [ ]:
import torch.nn as nn

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.L1 = nn.Conv2d(3,30,3,2,1)
        self.B1= nn.BatchNorm2d(30)
        self.L2 = nn.Conv2d(30,60,5,2,1)
        self.B2= nn.BatchNorm2d(60)
        self.L = nn.Linear(34560,10)

    def forward(self, x):
        #降二维，从start_dim开始相乘
        # x = torch.flatten(x,start_dim = 1)
        g1 = self.L1(x)
        g1 = self.B1(g1)
        # f1=torch.sigmoid(g1)
        f1=torch.relu(g1)

        g2 = self.L2(f1)
        g2 = self.B2(g2)
        f2 = torch.relu(g2)

        f2 = torch.flatten(f2,start_dim = 1)

        g = self.L(f2)

        return g
    
model = MyNet()

In [ ]:
import torchmetrics

lossf = nn.CrossEntropyLoss()     #loss函数
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  #SGD函数
metricsf = torchmetrics.Accuracy(task='multiclass', num_classes=10)       #torchmetrics函数

for i in range(100):
    for batchX, batchY in data_loader:
        score = model(batchX)
        score = torch.squeeze(score)
        loss = lossf(score, batchY)

        loss.backward() #求导
        optimizer.step() #梯度下降
        optimizer.zero_grad() #梯度清零

        mse = metricsf(score, batchY)    
    print(loss, metricsf.compute())
    metricsf.reset()


In [ ]:
torch.save(model, 'model3.pth')

In [ ]:
for batchX, batchY in data_loader2:
    score = model(batchX)
    score = torch.squeeze(score)
    mse = metricsf(score, batchY)  
      
print(metricsf.compute())